In [ ]:
!pip install --upgrade llama-index==0.9.14.post3 deeplake==3.8.8 openai==1.3.8 python-dotenv pandas==2.2.2

In [12]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [13]:
## load_document
from llama_index.schema import Document

with open("./data/zerodha_varsity_casestudy.txt",'r') as f:
    text = f.read()

chapters =[Document(text=i.strip(), metadata={"filename":i.strip()[:i.strip().find("\n")]+".txt" , "category": "finance"})  for i in text.split("###")]

/home/patelankit706/anaconda3/envs/llm/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.9) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [15]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(chunk_size=512, chunk_overlap=64)
node_parser = service_context.node_parser

nodes = node_parser.get_nodes_from_documents(chapters)

In [16]:
from llama_index.embeddings.openai import OpenAIEmbedding
embed_model = OpenAIEmbedding(
    model="text-embedding-ada-002",
)

In [17]:
from llama_index.vector_stores import DeepLakeVectorStore

my_activeloop_org_id = "patelankit706"
my_activeloop_dataset_name = "zerodha_varsity_qna"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False, embedding_function=embed_model)

Deep Lake Dataset in hub://patelankit706/zerodha_varsity_qna already exists, loading from the storage


In [18]:
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex

storage_context = StorageContext.from_defaults(vector_store=vector_store)
storage_context.docstore.add_documents(nodes)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

Uploading data to deeplake dataset.


100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.92s/it]
\

Dataset(path='hub://patelankit706/zerodha_varsity_qna', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
 embedding  embedding  (218, 1536)  float32   None   
    id        text      (218, 1)      str     None   
 metadata     json      (218, 1)      str     None   
   text       text      (218, 1)      str     None   


In [8]:
import time
from typing import List

from llama_index import (
    PromptTemplate,
    load_index_from_storage
)
from llama_index.llms.types import ChatMessage
from llama_index.llms import OpenAI
from llama_index.query_engine import CustomQueryEngine
from llama_index.retrievers import BaseRetriever
from pathlib import Path
from llama_index.chat_engine.condense_question import (
    CondenseQuestionChatEngine,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores import DeepLakeVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex

In [10]:
text2="""disabilities-policies
Zerodha has specific policies and procedures in place to accommodate individuals with disabilities. Here’s a summary of the key points related to account opening for visually impaired or illiterate persons:

1. Visually impaired and illiterate clients cannot open an account online.
2. New accounts cannot be opened if the PAN is not linked with Aadhaar. Even if Aadhaar is successfully linked with PAN, it must still be updated in the IT records, which can take up to 7 working days.
3. To open an account, the following documents must be filled out in the presence of a Zerodha representative:
   - Equity application form
   - Commodity application form
   - Declaration form
   - Vernacular risk disclosure document (RDD)
   - Notarised Demat Debit and Pledge Instruction (DDPI).
4. The thumb impression is used instead of signatures, with the left thumb for males and the right thumb for females.
5. Notary or doctor’s attestation is required on the KYC page and on the PAN and address proof.
6. The account opening charges are ₹500 towards Equity and Commodity trading account opening.
7. The account will be opened within 72 working hours once the forms are delivered to Zerodha’s head office.
For a detailed understanding of Zerodha’s policies and procedures, you can visit their official policies and procedures page. If you need further assistance or have specific requirements, it’s best to contact Zerodha directly to ensure all your needs are met."""